> #Note:
### 0. Rename this file to PA1yourLastName, e.g. PA1Chung, and work directly on it.
### 1. Due on 11:59 PM on Monday 3/14.
### 2. You may NOT use the backslash operator in Julia to solve the linear system, (A\b).  
### 3. You are encouraged to use codes from Lab.  
### 4. Indivisual work.  Copying in any form is a violation of the Honor code.
### 5. Present your answers and results in the TEXT enviornment and in complete sentences.

#Problem 1 (50 pts) 
In class, we go over how to formulate linear boundary value problems as linear system problems, and you may find the full discussion in Section 11.3.  In this problem, you need to implement ```LinearBVPsolver```. <br>
1a. Test your solver for Example 1 on page 688, and problem #2 in on page 689. <br>
1b. Do problem #8 in Section 11.3 on page 690.

In [42]:
#
# @param a <number> endpoint
# @param b <number> endpoint
# @param alpha <number> boundary condition
# @param beta <number> boundary condition
# @param p <function>
# @param q <function>
# @param r <function>
# @param h <function>
# @param N <number>
#
function LinearBVPsolver(a, b, alpha, beta, p, q, r, h, N)
    _a = zeros(N,1)
    _b = zeros(N,1)
    _c = zeros(N,1)
    _d = zeros(N,1)
    
    # step 1
    h = (b-a) / (N+1)
    x = a + h
    _a[1] = 2 + h^2 * q(x)
    _b[1] = -1 + (h/2) * p(x)
    _d[1] = -(h^2 * r(x)) + (1 + (h/2) * p(x)) * alpha
    
    # step 2
    for i = 2:N-1
        x = a + i*h
        _a[i] = 2 + h^2 * q(x)
        _b[i] = -1 + (h/2) * p(x)
        _c[i] = -1 - (h/2) * p(x)
        _d[i] = -(h^2) * r(x)
    end
    
    # step 3
    x = b - h
    _a[N] = 2 + h^2 * q(x)
    _c[N] = -1 - (h/2) * p(x)
    _d[N] = -(h^2) * r(x) + (1 - (h/2) * p(x)) * beta
    
    # Tridiagonal solver
    l = zeros(N,1)
    u = zeros(N,1)
    z = zeros(N,1)
    w = zeros(N,1)
    
    # step 4
    l[1] = _a[1]
    u[1] = _b[1]/_a[1]
    z[1] = _d[1]/l[1]
    
    # step 5
    for i = 2:N-1
        l[i] = _a[i] - _c[i]*u[i-1]
        u[i] = _b[i]/l[i]
        z[i] = (_d[i] - _c[i]*z[i-1]) / l[i]
    end
    
    # step 6
    l[N] = _a[N] - _c[N]*u[N-1]
    z[N] = (_d[N] - _c[N]*z[N-1]) / l[N]
    
    # step 7
    for i = N-1:-1:1
        w[i] = z[i] - u[i]*w[i+1]
    end
    
    # step 8
    for i = 0:N+1
        x = a + i*h
    end
    
    return x
end


LinearBVPsolver (generic function with 1 method)

In [46]:
# 1a - example 1
a = 1
b = 2
alpha = 1
beta = 2
p = x -> -2/x
q = x -> 2/x^2
r = x -> sin(log(x))/x^2
h = 0.1
N = 9
println(LinearBVPsolver(a, b, alpha, beta, p, q, r, h, N))

2.0


In [45]:
# 1a - #2
a = 0
b = pi/2
alpha = -0.3
beta = -0.1
p = x -> 1
q = x -> 2
r = x -> sin(x)

# a
h = pi/4
N = 9
println(LinearBVPsolver(a, b, alpha, beta, p, q, r, h, N))

# b
h = pi/8
N = 9
println(LinearBVPsolver(a, b, alpha, beta, p, q, r, h, N))

1.5707963267948966
1.5707963267948966


In [63]:
# 1b - page 690, #8

# problem params
S = 100 # 100lbs/in
Q = 200 # 200lbs/in^2
D = 8.8e7# 8.8 * 10^7
l = 50

# function arguments
a = 0
b = 50
alpha = 0
beta = 0
p = x -> 0
q = x -> S/D
r = x -> (Q/2*D)*x^2 - (Q*L/2*D)*x
h = 1
N = 50
println(LinearBVPsolver(a, b, alpha, beta, p, q, r, h, N))

LoadError: LoadError: UndefVarError: L not defined
while loading In[63], in expression starting on line 19

#Problem 2 (50 pts)
CG method only works for SPD matrices. The biconjugate gradient stabilized method (BiCGSTAB) is designed for general matrices (see the original paper http://epubs.siam.org/doi/abs/10.1137/0913035 for details if interested). In this problem, you are going to implement BiCGSTAB.  

To solve a linear system Ax = b, BiCGSTAB starts with an initial guess $x_0$ and proceeds as follows:
* $r_0 = b − Ax_0$
* Choose an arbitrary vector $\hat{r}_0$ such that $(\hat{r}_0, r_0) \ne 0$, e.g., $\hat{r}_0 = r_0$.
* $\rho_0 = \alpha = \omega_0 = 1$
* $v_0 = p_0 = 0$
* For i = 1, 2, 3, …
  * $\rho_i = (\hat{r}_0, r_{i−1})$
  * $\beta = (\rho_i/\rho_{i−1})(\alpha/\omega_{i−1})$
  * $p_i = r_{i−1} + \beta(p_{i−1} − \omega_{i−1}v_{i−1})$
  * $v_i = Ap_i$
  * $\alpha = \rho_i/(\hat{r}_0, v_i)$
  * $s = r_{i−1} − \alpha v_i$
  * if $\|s\|_{\infty}$ sufficiently small, then set $x_i = x_{i−1} + \alpha p_i$ and quit
  * $t = As$
  * $\omega_i = (t, s)/(t, t)$
  * $x_i = x_{i−1} + \alpha p_i + \omega_i s$
  * $r_i = s − \omega_i t$
  
>  $(u, v)$ means dot product of vector u and v. The algorithm is from https://en.wikipedia.org/wiki/Biconjugate_gradient_stabilized_method


Do the following: <br>
2a. Implement BiCGSTAB and test it for #4ab in Section 6.1 on page 368. <br>
2b. Use BiCGSTAB with TOL=1e-8 to solve the system $Ax = b$, where $A$ is defined as 
<img src="PA1Prob2Mat.png">
and $b=(1, 0, 0, ..., 0)$, and $n=101$ (the code for creating this matrix is given).   <br>
2c. Use SOR with TOL=1e-8 and w=1.9 to solve the same system. Compare number of iterations required in 2b and 2c.  What do you observe?

> Note that the matrix is NOT tridiagonal, nor SPD.

In [61]:
function BiCGSTAB(A, b, x, TOL)
    n, n1 = size(A)
    r = b - A*x
    r_hat = r
    rho = alpha = omega = 1
    v = p = 0
    _r = zeros(n)
    for i=1:n
        _rho = rho
        rho = dot(r_hat, r)
        beta = (rho/_rho) * (alpha/omega)
        _r[1:n] = r[1:n]
        p = _r + beta * (p - omega * v)
        v = A * p
        alpha = rho / dot(r_hat, v)
        s = _r - alpha*v
        _x = x
        if norm(s, Inf) < TOL
            x = x + a*p
            return x
        end
        t = A*s
        omega = dot(t,s) / dot(t,t)
        x = x + a*p + omega*s
        r = s - omega*t
    end
    return x
end

BiCGSTAB (generic function with 1 method)

In [48]:
function genTriDiag(n, l, d, r)
    #generate a tridiagonal matrix of size n by n with entries l, d, r.
    T = zeros(n,n)
    for i = 1:n
        T[i,i] = d
        if i+1 <= n
            T[i, i+1] = r
        end
        if i-1 >= 1
            T[i, i-1] = l
        end
    end
    return T
end

genTriDiag (generic function with 1 method)

In [62]:
A = zeros(101,101)
A[1:100,1:100] = genTriDiag(100, 1, -2, 1)
A[100, 101] = 1
A[101, 1] = 4
A[101, 2] = -1.0
g = 0.1
A[101, 99] = g
A[101,100] = -4*g
A[101,101] = 3*g
b = zeros(101)
b[1] = 1
BiCGSTAB(A,b,zeros(101), 1e-8)

101-element Array{Float64,1}:
    -1.13087 
    -4.31125 
    -6.83335 
    -9.95346 
    -9.87865 
   -36.4638  
    12.1961  
     3.2896  
    13.4126  
    -4.92378 
    -6.95528 
    -0.315984
    34.5113  
     ⋮       
  6425.95    
  7655.38    
  9121.44    
 10889.4     
 13100.9     
 15800.2     
 19132.4     
 23037.1     
 27775.7     
 33516.6     
 40488.0     
 48966.5     